# Lecture 7: AI Notetaker - Building Blocks

Link to google colab: https://colab.research.google.com/drive/1PKVKD6pSm5e-HW1dnqmkygtfXVHv13zH?usp=sharing

We are using these. they are installed already: -

* pip install -q torch transformers bitsandbytes huggingface_hub gradio

In [1]:
# Load imports and get the hgginface token. 
import os
import torch
from huggingface_hub import login
from dotenv import load_dotenv

load_dotenv()

hf_token = os.getenv("HUGGINGFACE_API_KEY")
OPENAI_KEY = os.getenv("OPENAI_API_KEY")
login(hf_token, add_to_git_credential=True)

In [ ]:
from transformers import pipeline, BitsAndBytesConfig
# use the pipeline for efficiency. Still getting KeyError: 'layout'. Assume it is a memory issue. 
# The instruct version of this model is fine tuned for following instructions like a typical chat experience. 
chatbot = pipeline(
    model="meta-llama/Llama-3.1-8B-Instruct",
    torch_dtype=torch.bfloat16,     # 
    device_map="auto",              # Chooses GPU if there, or CP
    model_kwargs={"quantization_config": BitsAndBytesConfig(load_in_4bit=True)},    # shrink the RAM requirement for the model by making it 4bit
    # Watch out for pad_token_id=128001. This is particularly for the model meta-llama/Llama-3.1-8B-Instruct.
    # If you were to choose another model, you will have to get the correct eos_token_id for that model.
    # This is the token id of the end of sentence token. This gets rid of an annoying error that pops up. 
    pad_token_id=128001
    )

In [3]:
# Create the message
prompt = "What is a current existential crisis faced by humanity?"

messages = [
    {'role':'user','content':prompt}
]
# and view
messages

[{'role': 'user',
  'content': 'What is a current existential crisis faced by humanity?'}]

In [ ]:
# Get the response and print. Limit to 50 tokens to simplify
response = chatbot(messages, max_new_tokens=50)
print(response[0]['generated_text'][-1]['content'])

In [ ]:
# now create a chat bot. 
messages = []
user_input= ""

while user_input != 'bye':
  user_input = str(input("You: "))  # get user input
  messages.append({'role':'user', 'content':user_input})  # append it to the list with the role of user
  ai_response = chatbot(messages, max_new_tokens=50)  # generate the response.
  # and up pick the final response. [0] gets us the entire response from the outer list. 
  # ['generated_text'] gets us the generated_text from the inner list
  # [-1] gets us the final AI response from all the data with ['content'] returning the specific content of that response. 
  ai_response = ai_response[0]['generated_text'][-1]['content']
  print("AI: ", ai_response) # Dipslay it
  messages.append({'role':'assistant','content':ai_response}) # and append it to the messages so we can pass in the full context next time around

"""
An example of how this might look: -

    You: hi there
    AI:  How are you today? Is there something I can help you with or would you like to chat?
    You: what are you capable of?
    AI:  I'm a large language model, I can perform a wide range of tasks, including:

    1. **Answering questions**: I can provide information on various topics, from science and history to entertainment and culture.
    2. **Generating text**: I can
    You: continue your sentence
    AI:  ...generate text on a given topic, summarize long pieces of text, create content, and even assist with writing tasks like proofreading and editing.

    3. **Translation**: I can translate text from one language to another, helping to bridge language gaps.

    You: bye
    AI:  Bye! It was nice chatting with you. If you need anything else, feel free to come back anytime. Have a great day!
"""

In [ ]:
# Repeat the above but this time print what is being passed in
from pprint import pprint
messages = []
user_input = ''

while user_input != 'bye':
  user_input = input('You: ')
  messages.append({'role':'user','content':user_input})
  print('\nWhat is being passed to the model?\n')
  pprint(messages)
  print('\n')
  assistant = chatbot(messages)[0]['generated_text'][-1]['content']
  print("AI: ", assistant)
  messages.append({'role':'assistant','content':assistant})

"""
It looks like this: -

    You: hi there

    What is being passed to the model?

    [{'content': 'hi there', 'role': 'user'}]


    AI:  How are you today? Is there something I can help you with or would you like to chat?
    You: what time in the morning is best for working out?

    What is being passed to the model?

    [{'content': 'hi there', 'role': 'user'},
    {'content': 'How are you today? Is there something I can help you with or '
                'would you like to chat?',
    'role': 'assistant'},
    {'content': 'what time in the morning is best for working out?',
    'role': 'user'}]


    AI:  Research suggests that the best time for a workout is early in the morning, typically between 5-
    You: please continue your thought

    What is being passed to the model?

    [{'content': 'hi there', 'role': 'user'},
    {'content': 'How are you today? Is there something I can help you with or '
                'would you like to chat?',
    'role': 'assistant'},
    {'content': 'what time in the morning is best for working out?',
    'role': 'user'},
    {'content': 'Research suggests that the best time for a workout is early in '
                'the morning, typically between 5-',
    'role': 'assistant'},
    {'content': 'please continue your thought', 'role': 'user'}]


    AI:  ...7 am. This is often referred to as the "golden hour" for working out. Here
    You: bye

    What is being passed to the model?

    [{'content': 'hi there', 'role': 'user'},
    {'content': 'How are you today? Is there something I can help you with or '
                'would you like to chat?',
    'role': 'assistant'},
    {'content': 'what time in the morning is best for working out?',
    'role': 'user'},
    {'content': 'Research suggests that the best time for a workout is early in '
                'the morning, typically between 5-',
    'role': 'assistant'},
    {'content': 'please continue your thought', 'role': 'user'},
    {'content': '...7 am. This is often referred to as the "golden hour" for '
                'working out. Here',
    'role': 'assistant'},
    {'content': 'bye', 'role': 'user'}]


    AI:  Have a great day!

"""  

In [ ]:
# The way gradio works is that you need to craft a function that meets the requirements for its out of the box interface
# messages is the new message entered by the user. We build up the history by adding the messages to the history. 
# we use the model to generate a response and return it (the full response i.e. 
# {'content': 'How are you today? Is there something I can help you with or would you like to chat?', 'role': 'assistant'})
# I am guessing the gradio adds this to the history.
def chat(messages, history):
  history.append({'role':'user','content':messages})
  ai_response = chatbot(history, max_new_tokens=50)[0]['generated_text'][-1]
  return ai_response

In [ ]:
# now use gradio to display a better interface.
import gradio as gr

demo = gr.ChatInterface(
    fn = chat,
    type="messages"
)

demo.launch(debug=True) # If you get an error, adding debug=True helps you resolve it.